4.2. Реализовать метод стрельбы и конечно-разностный метод решения краевой задачи для ОДУ в виде программ. С использованием разработанного программного обеспечения решить краевую задачу для обыкновенного дифференциального уравнения 2-го порядка на указанном отрезке. Оценить погрешность численного решения с использованием метода Рунге – Ромберга и путем сравнения с точным решением. 

Варинаит 23

Краевая задача $$ x(x^2 + 6)y'' - 4(x^2 + 3)y' +6xy = 0$$ $$y'(0) = 0 $$ $$y(4) - y'(4) = 26 $$
Точное решение $$y(x) = x^3 + x^2 + 2$$

Ввразим $y''$
$$y'' = \frac{4y'(x^2 + 3)}{x(x^2 + 6)} - \frac{6y}{x^2 + 6}$$

In [ ]:
import typing

In [1]:
a, b = 0, 4
h = 0.1

In [4]:
def f11 (x:float, y:float, y1:float) -> float:
    return 4 * y1 * (x**2 + 3) / x / (x**2 + 6) - 6 * y /(x**2 + 6) 

def frange(start:float, stop:float, step:float) -> float:
    while start < stop:
        yield start
        start += step

def runge_knutta_4(x:float, y0:float, z0:float, f:typing.Callable[[float, float, float], float], h:float) -> (float, float):
    K1 = h * z0
    L1 = h * f(x, y0, z0)
    K2 = h * (z0 + L1 / 2)
    L2 = h * f(x + h / 2, y0 + K1 / 2, z0 + L1 / 2)
    K3 = h * (z0 + L2/2) 
    L3 = h * f(x + h / 2, y0 +  K2 / 2, z0 + L2 / 2)
    K4 = h * (z0 + L3/2)
    L4 = h * f(x + h / 2, y0 +  K3 / 2, z0 + L3 / 2)


    delta_y =  (K1 + 2* K2 + 2 * K3 + K4) / 6
    delta_z =  (L1 + 2* L2 + 2 * L3 + L4) / 6
    return z0 + delta_z, y0 + delta_y

def func(a:float, b:float, z:float, y:float, step:float, f11:typing.Callable, fun:typing.Callable) -> list:
    data = [y]
    for x in frange(a, b, step):
        z, y = fun(x, y, z, f11, step)
        data.append(y)
    return data

def derivative(x:list, y:list, x0:float):
    i = 0
    while i < len(x) - 1 and x[i + 1] < x0:
        i += 1
    return (y[i + 1] - y[i]) / (x[i + 1] - x[i])

In [5]:
def shooting(a:float, b:float, h:float, y0:float, y1:float, eps:float, f:typing.Callable) -> list:
    etta1, etta2 = 1,  0.8

    phi1 = func(a, b, etta1, y0, h, f, runge_knutta_4)[-1] - y1
    phi2 = func(a, b, etta2, y0, h, f, runge_knutta_4)[-1] - y1
    error = 10 * eps
    while error > eps:
        nu1, nu2 = nu2, nu2 - phi2 * (nu2 - nu1) / (phi2 - phi1)
        phi1, phi2 = phi2, func(a, b, nu2, y0, h, f, runge_knutta_4)[-1] - y1
        error = abs(phi1 - phi2)

    return func(a, b, nu2, y0, h, f, runge_knutta_4)